In [29]:
from collections import OrderedDict

import numpy as np
import pandas as pd
import altair as alt
from numpy.random import Generator, PCG64

In [43]:
def generate_test_data(n):
    x = np.linspace(start=0, stop=1, num=n)
    t = np.sin(2 * np.pi * x)

    return pd.DataFrame({'x': x, 't': t})

def generate_training_data(n, seed=None):
    rg = Generator(PCG64(seed))

    x = np.linspace(start=0, stop=1, num=n)
    t = np.sin(2 * np.pi * x) + rg.normal(scale=0.2, size=n)

    return pd.DataFrame({'x': x, 't': t})

In [3]:
chart_train = alt.Chart(df_train).mark_point(color='blue').encode(
    x='x',
    y='t',
)

chart_test = alt.Chart(df_test).mark_line(color='lime').encode(
    x='x',
    y='t',
)

chart_train + chart_test

alt.LayerChart(...)

In [4]:
def polynomial(x_train, t_train, power, return_w=False):
    X_train = np.stack([np.power(x_train, p) for p in range(power+1)], axis=1)
    w_ml = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ t_train

    def _polynomial(x_test):
        X_test = np.stack([np.power(x_test, p) for p in range(power+1)], axis=1)
        return X_test @ w_ml
    
    if return_w:
        return _polynomial, w_ml
    else:
        return _polynomial

In [5]:
power = 3

f = polynomial(x_train, t_train, power=power)

data = pd.DataFrame({
    'x': x_test,
    't': f(x_test),
})

chart_pred = alt.Chart(data).mark_line(color='red').encode(
    x='x',
    y='t',
)

chart_train + chart_test + chart_pred

alt.LayerChart(...)

In [7]:
def rms(f, x_test, t_test):
    return np.sqrt(np.mean((f(x_test) - t_test)**2))

powers = list(range(10))

df_rms_train = pd.DataFrame({
    'power': powers,
    'rms': [rms(polynomial(x_train, t_train, power), x_train, t_train) for power in powers]})

df_rms_test = pd.DataFrame({
    'power': powers,
    'rms': [rms(polynomial(x_train, t_train, power), x_test, t_test) for power in powers]})

chart_rms_train = alt.Chart(df_rms_train).mark_line(point=True, color='blue').encode(
    x='power',
    y='rms')

chart_rms_test = alt.Chart(df_rms_test).mark_line(point=True, color='red').encode(
    x='power',
    y='rms')

chart_rms_train + chart_rms_test


alt.LayerChart(...)

In [8]:
df_coefficients = pd.concat([pd.DataFrame({
    f'M={power}': polynomial(x_train, t_train, power=power, return_w=True)[-1]
}) for power in [0, 1, 6, 9]], axis=1)

df_coefficients

,M=0,M=1,M=6,M=9
0,-0.039288,0.501583,0.062194,0.048551
1,NaN,-1.081742,-0.687544,-17.140434
2,NaN,NaN,61.121484,566.069995
3,NaN,NaN,-279.282402,-5883.051116
4,NaN,NaN,455.109969,31048.778856
5,NaN,NaN,-319.314197,-93178.434412
6,NaN,NaN,83.071011,164491.118910
7,NaN,NaN,NaN,-168794.264626
8,NaN,NaN,NaN,93095.820038
9,NaN,NaN,NaN,-21328.855448


In [15]:
np.random.seed(34)

x_train_n15 = np.linspace(0, 1, 15)
t_train_n15 = np.sin(2 * np.pi * x_train_n15) + np.random.normal(scale=0.2, size=15)

x_train_n100 = np.linspace(0, 1, 100)
t_train_n100 = np.sin(2 * np.pi * x_train_n100) + np.random.normal(scale=0.2, size=100)

x_test = np.linspace(0, 1, 100)
t_test = np.sin(2 * np.pi * x_test)

df_train_n15 = pd.DataFrame({'x': x_train_n15, 't': t_train_n15})
df_train_n100 = pd.DataFrame({'x': x_train_n100, 't': t_train_n100})
df_test = pd.DataFrame({'x': x_test, 't': t_test})

In [22]:
power = 9

f = polynomial(x_train_n100, t_train_n100, power=power)

data = pd.DataFrame({
    'x': x_test,
    't': f(x_test),
})

chart_train = alt.Chart(df_train_n100).mark_point(color='blue').encode(
    x='x',
    y='t',
)

chart_pred = alt.Chart(data).mark_line(color='red').encode(
    x='x',
    y='t',
)

chart_train + chart_test + chart_pred

alt.LayerChart(...)

Docstring:
normal(loc=0.0, scale=1.0, size=None)

Draw random samples from a normal (Gaussian) distribution.

The probability density function of the normal distribution, first
derived by De Moivre and 200 years later by both Gauss and Laplace
independently [2]_, is often called the bell curve because of
its characteristic shape (see the example below).

The normal distributions occurs often in nature.  For example, it
describes the commonly occurring distribution of samples influenced
by a large number of tiny, random disturbances, each with its own
unique distribution [2]_.

Parameters
----------
loc : float or array_like of floats
    Mean ("centre") of the distribution.
scale : float or array_like of floats
    Standard deviation (spread or "width") of the distribution. Must be
    non-negative.
size : int or tuple of ints, optional
    Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
    ``m * n * k`` samples are drawn.  If size is ``None`` (default),
    a single v